<h1>CSE-6363 Machine Learning, Fall 2022<br>Programming Assignment 2<h1>



In [7]:
# importing all the required libraries
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummaryX import summary as summaryX
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# checking requirements
print(f'Pytorch version: {torch.__version__}')
print(f'cuda version: {torch.version.cuda}')


Pytorch version: 1.12.1+cpu
cuda version: None


In [3]:
# defining the LeNet-5 architecture
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.convolution1 = nn.Conv2d(in_channels=1, out_channels=6,
                                      kernel_size=5, stride=1)
        self.convolution2 = nn.Conv2d(in_channels=6, out_channels=16,
                                      kernel_size=5, stride=1)
        self.convolution3 = nn.Conv2d(in_channels=16, out_channels=120,
                                      kernel_size=5, stride=1)
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
        self.tanh = nn.Tanh()
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.convolution1(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.convolution2(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.convolution3(x)
        x = self.tanh(x)

        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.tanh(x)
        x = self.linear2(x)
        return x


model = LeNet()
x = torch.randn(64,1,32,32)
output = model(x)

print(model)
summary(model, (1,32,32))
print("output.shape : ",output.shape)


LeNet(
  (convolution1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (convolution2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (convolution3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
       

In [4]:
learning_rate = 0.001


train_dataset = datasets.MNIST(root='mnist_data', train=True,
                               transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]), download=True)
test_dataset = datasets.MNIST(root='mnist_data', train=False,
                              transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)
dataset_size = {'train': len(train_dataset), 'test': len(test_dataset)}
print(dataset_size)


{'train': 60000, 'test': 10000}


In [5]:

#display bar for progress
class progress_bar(object):

    disp_style = """
        <p>Loss: {loss:0.4f}   {value} / {length}</p>
        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
    """

    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)

    def html(self, count, loss):
        return HTML(self.disp_style.format(length=self.length, value=count, loss=loss))

    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

#displaying strucure
#net = LeNet()
#print(net)

model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


def train_model(model, criterion, optimizer, no_epochs, dataloaders, dataset_size, starting_epoch=1):

    plot_training_loss = []
    plot_validation_loss = []

    for epoch in range(starting_epoch, no_epochs):
        print()
        print('Epoch', epoch)
        running_loss = 0.0
        validation_loss = 0.0

        # train phase
        model.train()

        # create a progress bar
        progress = progress_bar(length=dataset_size["train"])

        for data in dataloaders[0]:
            # Move the training data to the GPU
            inputs, labels = data
            batch_size = inputs.shape[0]

            # clear previous gradient computation
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.data * batch_size
            # update progress bar
            progress.update(batch_size, running_loss)

        epoch_loss = running_loss / dataset_size["train"]
        print('Training loss:', epoch_loss.item())

        plot_training_loss.append(epoch_loss)

        # validation phase
        model.eval()
        with torch.no_grad():
            for data in dataloaders[-1]:
                inputs, labels = data
                batch_size = inputs.shape[0]

                output = model(inputs)

                # calculate the loss
                optimizer.zero_grad()
                loss = criterion(output, labels)

                # update running loss value
                validation_loss += loss.data * batch_size

        epoch_validation_loss = validation_loss / dataset_size["test"]
        print('Validation loss:', epoch_validation_loss.item())
        plot_validation_loss.append(epoch_validation_loss)

    return plot_training_loss, plot_validation_loss, model


if __name__ == "__main__":
    train_losses, validation_losses, model = train_model(model=model, criterion=criterion, optimizer=optimizer,
                                                         no_epochs=15, dataloaders=[train_loader, test_loader],
                                                         dataset_size=dataset_size)



Epoch 1


Training loss: 0.2816525101661682
Validation loss: 0.10849947482347488

Epoch 2


Training loss: 0.09355814754962921
Validation loss: 0.0781828761100769

Epoch 3


Training loss: 0.06409594416618347
Validation loss: 0.05717148259282112

Epoch 4


Training loss: 0.04636077582836151
Validation loss: 0.055973779410123825

Epoch 5


...

Training loss: 0.018321217969059944
Validation loss: 0.046532291918992996

Epoch 11


Training loss: 0.014198300428688526
Validation loss: 0.043747592717409134

Epoch 12


Training loss: 0.012985846027731895
Validation loss: 0.048601578921079636

Epoch 13


Training loss: 0.011250624433159828
Validation loss: 0.04877966269850731

Epoch 14


Training loss: 0.010928183794021606
Validation loss: 0.05842728167772293


In [6]:
# function for Accuracy
def val(loader, model, train=True):
    correct_samples = total_samples = 0
    model.eval()
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            batch_size = inputs.shape[0]

            output = model(inputs)
            _, prediction = output.max(1)
            correct_samples += (prediction == labels).sum()
            total_samples += prediction.size(0)
    val = (correct_samples.item()/total_samples)*100
    if train:
        print("Model Predicted {} correctly out of {} from training dataset, Acuracy : {:.2f}".format(
            correct_samples.item(), total_samples, val))
    else:
        print("Model Predicted {} correctly out of {} from testing dataset, Acuracy : {:.2f}".format(
            correct_samples.item(), total_samples, val))
    model.train()


val(train_loader, model)
val(test_loader, model, train=False)


Model Predicted 59678 correctly out of 60000 from training dataset, Acuracy : 99.46
Model Predicted 9847 correctly out of 10000 from testing dataset, Acuracy : 98.47
